In [ ]:
!pip install awswrangler
!pip install unidecode

In [ ]:
import awswrangler as wr
import pandas as pd
from unidecode import unidecode
from datetime import date

In [ ]:
input_paths = [
    's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_geo_mappings/population_demographics/us/ACSST5Y2022.S0101-Data.csv',
    's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_geo_mappings/zipcode_to_zcta/ZIPCodetoZCTACrosswalk2022UDS.csv'
]
output_paths = [
    's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_geo_mappings/population_demographics/output/us/',
    's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_geo_mappings/outputs/zcta_to_zip/',
]
table_names = ['mars_population_demographics_us', 'mars_zip_zcta_crosswalk']
target_database = 'sandbox_measurement_analyst'
target_workgroup = 'cmg-oasis-prod-measurement_analyst-wkgrp'
today_for_path = date.today().strftime("%Y%m%d")
today_for_tbl = date.today()

In [ ]:
for source, tbl, target in zip(input_paths, table_names, output_paths):
    if tbl == 'mars_population_demographics_us':
        column_names = [
            "GEO_ID",
            "NAME",
            "S0101_C01_001E",  # TOTAL
            "S0101_C01_023E",  # 18-24
            "S0101_C01_007E",  # 25-29
            "S0101_C01_008E",  # 30-34
            "S0101_C01_009E",  # 35-39
            "S0101_C01_010E",  # 40-44
            "S0101_C01_011E",  # 45-49
            "S0101_C03_023E",  # 18-24
            "S0101_C03_007E",  # 25-29
            "S0101_C03_008E",  # 30-34
            "S0101_C03_009E",  # 35-39
            "S0101_C03_010E",  # 40-44
            "S0101_C03_011E",  # 45-49
            "S0101_C05_023E",  # 18-24
            "S0101_C05_007E",  # 25-29
            "S0101_C05_008E",  # 30-34
            "S0101_C05_009E",  # 35-39
            "S0101_C05_010E",  # 40-44
            "S0101_C05_011E",  # 45-49
            "S0101_C01_012E", #50-54,
            "S0101_C01_013E", #55-59,
            "S0101_C01_014E", #60-64,
            "S0101_C01_015E", #65-69,
            "S0101_C01_016E", #70-74,
            "S0101_C01_017E", #75-79,
            "S0101_C01_018E", #80-84,
            "S0101_C01_019E", #85+
        ]
        columns_to_convert = [
            "S0101_C01_001E",
            "S0101_C01_023E",
            "S0101_C01_007E",
            "S0101_C01_008E",
            "S0101_C01_009E",
            "S0101_C01_010E",
            "S0101_C01_011E",
            "S0101_C03_023E",
            "S0101_C03_007E",
            "S0101_C03_008E",
            "S0101_C03_009E",
            "S0101_C03_010E",
            "S0101_C03_011E",
            "S0101_C05_023E",
            "S0101_C05_007E",
            "S0101_C05_008E",
            "S0101_C05_009E",
            "S0101_C05_010E",
            "S0101_C05_011E",
            "S0101_C01_012E",
            "S0101_C01_013E",
            "S0101_C01_014E",
            "S0101_C01_015E",
            "S0101_C01_016E",
            "S0101_C01_017E",
            "S0101_C01_018E",
            "S0101_C01_019E"
        ]    
        df = pd.read_csv(source, usecols=column_names, header=0, skiprows=[2], low_memory=False)
        df['etl_load_date'] = today_for_tbl
        df.rename(columns={'NAME': 'zcta'}, inplace=True)
        df['zcta'] = df['zcta'].str.slice(6, 11)
        df = df[(df['GEO_ID'] != 'Geography') & (df['zcta'].str.contains(r'^\d+$'))]
        for column in columns_to_convert:
            df[column] = pd.to_numeric(df[column], errors='coerce')     
    else:
        col_names = ['ZIP_CODE', 'zcta', 'STATE']
        df = pd.read_csv(source, usecols=col_names, header=0, dtype='str')
        df.rename(columns={'ZIP_CODE': 'zip', 'zcta': 'zcta', 'STATE': 'state'}, inplace=True)
        df['etl_load_date'] = today_for_tbl
    
    new_path = f'{target}{today_for_path}/'
    
    try: 
        wr.s3.to_parquet(
            df=df,
            path=new_path,
            dataset=True,
            database=target_database,
            table=tbl,
            mode="overwrite",
            index=False
        )
    except Exception as e:
        raise e

In [ ]:
new_path = 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_geo_mappings/population_demographics/output/us_islands/'
target_database = 'sandbox_measurement_analyst'
target_workgroup = 'cmg-oasis-prod-measurement_analyst-wkgrp'
table_name = 'mars_population_demographics_us_islands_raw'

file_paths = [
    's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_geo_mappings/population_demographics/us_islands/DECENNIALDPGU2020.DP1-Data.csv',
    's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_geo_mappings/population_demographics/us_islands/DECENNIALDPMP2020.DP1-Data.csv',
    's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_geo_mappings/population_demographics/us_islands/DECENNIALDPVI2020.DP1-Data.csv',
    's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_geo_mappings/population_demographics/us_islands/DECENNIALDPAS2020.DP1-Data.csv'
]

column_names = [
    "NAME",
    "DP1_0001C", 
    "DP1_0006C",
    "DP1_0007C",
    "DP1_0008C",
    "DP1_0009C",
    "DP1_0010C",
    "DP1_0011C",
    "DP1_0012C",
    "DP1_0013C",
    "DP1_0014C",
    "DP1_0015C",
    "DP1_0016C",
    "DP1_0017C",
    "DP1_0018C",
    "DP1_0019C",
    "DP1_0054C",
    "DP1_0055C",
    "DP1_0056C",
    "DP1_0057C",
    "DP1_0058C",
    "DP1_0059C"
]

dfs = []
for file in file_paths:
    df = pd.read_csv(file, usecols = column_names, header=0, skiprows=[1])
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index = True)
combined_df['NAME'] = combined_df['NAME'].apply(unidecode)

try: 
    wr.s3.to_parquet(
            df=combined_df,
            path=new_path,
            dataset=True,
            database=target_database,
            table=table_name,
            mode="overwrite",
            index=False
        )
except Exception as e:
    raise e